In [1]:
import requests
import json

In [24]:
class GitHubRepository:
    def __init__(self, name, owner,  description, homepage, license, forks, watchers):
        self.name = name
        self.owner = owner
        self.description = description
        self.homepage = homepage
        self.license = license  # License could be another class with its own attributes
        self.forks = forks
        #self.date_of_collection = date_of_collection
        self.watchers = watchers

    def __str__(self):
        return f"{self.owner}/{self.name}: license = ({self.license}) forks = ({self.forks}) watchers = ({self.watchers})"

In [3]:
class PullRequest:

  def parse(self, dict):
    self.title = dict['title'] if 'title' in dict else None
    self.number = dict['number'] if 'number' in dict else None
    self.body = dict['body'] if 'body' in dict else None
    self.state = dict['state'] if 'state' in dict else None
    self.created_at = dict['created_at'] if 'created_at' in dict else None
    self.user = dict['user']['login'] if 'user' in dict and 'login' in dict['user'] else None
    self.closed_at = dict['closed_at'] if 'closed_at' in dict else None

  def __str__(self):
    return f"{self.user}/{self.title}: ({self.state}) ({self.number}) ({self.created_at}) ({self.closed_at})"

In [22]:
class PRStatus:
  def __init__(self,number,commits,additions,deletions,changed_files):
    self.number = number
    self.commits = commits
    self.additions = additions
    self.deletions = deletions
    self.changed_files = changed_files

  @staticmethod
  def parse(dict):
    number = dict['number'] if 'number' in dict else None
    commits = dict['commits'] if 'commits' in dict else None
    additions = dict['additions'] if 'additions' in dict else None
    deletions = dict['deletions'] if 'deletions' in dict else None
    changed_files = dict['changed_files'] if 'changed_files' in dict else None

    return PRStatus(number,commits,additions,deletions,changed_files)

  def __str__(self):
    return f"{self.number}: commits = ({self.commits}) additions = ({self.additions}) deletions = ({self.deletions}) file changes = ({self.changed_files})"

In [18]:
class UserInfo:
  def __init__(self, name, no_of_repos, followers, following):
    self.name = name
    self.no_of_repos = no_of_repos
    self.followers = followers
    self.following = following

  @staticmethod
  def parse(dict):
    name = dict['name'] if 'name' in dict else None
    no_of_repos = dict['public_repos'] if 'public_repos' in dict else None
    followers = dict['followers'] if 'followers' in dict else None
    following = dict['following'] if 'following' in dict else None

    return UserInfo(name, no_of_repos, followers, following)

  def __str__(self):
    return f'{self.name}: repos = {self.no_of_repos}, followers = {self.followers}, following = {self.following}'

In [26]:
owner = "debsourav33"
repo_name = "Leetcode-Solutions"

def get_repository(owner, repo) -> GitHubRepository :

  url = f"https://api.github.com/repos/{owner}/{repo}"
  response = requests.get(url)

  if response.status_code != 200:
    return None

  resp_dict = response.json()
  #print(resp_dict)

  name = resp_dict['name'] if 'name' in resp_dict else None
  owner = resp_dict['owner']['login'] if 'owner' in resp_dict and 'login' in resp_dict['owner'] else None
  description = resp_dict['description'] if 'description' in resp_dict else None
  forks = resp_dict['forks'] if 'forks' in resp_dict else None
  watchers = resp_dict['watchers'] if 'watchers' in resp_dict else None
  license = resp_dict['license']['name'] if 'license' in resp_dict and resp_dict['license'] != None and 'name' in resp_dict['license'] else None

  pretty_response = json.dumps(resp_dict, indent = 4)
  #print(pretty_response)

  return GitHubRepository(name, owner, description, None, license, forks, watchers)
  #print(pretty_response)

github_repo = get_repository('ramnes', 'notion-sdk-py')
print(github_repo)

ramnes/notion-sdk-py: license = (MIT License) forks = (114) watchers = (1371)


In [23]:
#-Pull Request

owner = "jabref"
repo = "jabref"

def getPullRequests(owner, repo):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
  print(url)
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Status code: {response.status_code}")
      return None
  else:
    resp_list = response.json()
    #print(len(resp_list))
    ret = []
    for item in resp_list:
      pr= PullRequest()
      pr.parse(item)
      ret.append(pr)
      #print(pr)
    return ret


def getPullRequestInfo(owner, repo, number):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{number}"
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Failed to retrieve pull requests. Status code: {response.status_code}")
      return None
  else:
    resp_dict = response.json()
    #print(type(resp_dict))
    #print(resp_dict)
    pr_info = PRStatus.parse(resp_dict)
    #print(pr_info)
    return pr_info


pull_requests = getPullRequests(owner,repo)
for pr in pull_requests:
  if pr.number != None:
    pr_info = getPullRequestInfo(owner, repo, pr.number)
    print(pr_info)


https://api.github.com/repos/jabref/jabref/pulls
10620: commits = (7) additions = (70) deletions = (9) file changes = (10)
10610: commits = (4) additions = (167) deletions = (91) file changes = (12)
10592: commits = (29) additions = (474) deletions = (13) file changes = (17)
10591: commits = (22) additions = (142) deletions = (30) file changes = (10)
10586: commits = (36) additions = (2837) deletions = (24) file changes = (47)
10582: commits = (13) additions = (38) deletions = (4) file changes = (4)
10578: commits = (12) additions = (222) deletions = (174) file changes = (12)
10577: commits = (2) additions = (77) deletions = (22) file changes = (4)
10565: commits = (7) additions = (29) deletions = (0) file changes = (2)
10541: commits = (3) additions = (56158) deletions = (3) file changes = (5)
10540: commits = (5) additions = (254) deletions = (13) file changes = (4)
10526: commits = (10) additions = (94) deletions = (6) file changes = (4)
10521: commits = (9) additions = (73) deletio

In [21]:
user = "jabref"


def getUserInfo(user):
  url = f'https://api.github.com/users/{user}'

  response = requests.get(url)

  if response.status_code != 200:
    print(f"Status code: {response.status_code}")
    return None
  else:
    resp_dict = response.json()
    print(json.dumps(resp_dict,indent = 4))
    user_info = UserInfo.parse(resp_dict)
    print(user_info)


getUserInfo(user)


{
    "login": "JabRef",
    "id": 3914421,
    "node_id": "MDEyOk9yZ2FuaXphdGlvbjM5MTQ0MjE=",
    "avatar_url": "https://avatars.githubusercontent.com/u/3914421?v=4",
    "gravatar_id": "",
    "url": "https://api.github.com/users/JabRef",
    "html_url": "https://github.com/JabRef",
    "followers_url": "https://api.github.com/users/JabRef/followers",
    "following_url": "https://api.github.com/users/JabRef/following{/other_user}",
    "gists_url": "https://api.github.com/users/JabRef/gists{/gist_id}",
    "starred_url": "https://api.github.com/users/JabRef/starred{/owner}{/repo}",
    "subscriptions_url": "https://api.github.com/users/JabRef/subscriptions",
    "organizations_url": "https://api.github.com/users/JabRef/orgs",
    "repos_url": "https://api.github.com/users/JabRef/repos",
    "events_url": "https://api.github.com/users/JabRef/events{/privacy}",
    "received_events_url": "https://api.github.com/users/JabRef/received_events",
    "type": "Organization",
    "site_admin